# Solving PDE IVPs

In our last class we explored relaxation methods for solving PDE boundary value problems. Now it is time to turn our attention to **initial value problems (IVPs)**, and some classic 1D PDEs!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## The diffusion equation 

The one-dimensional diffusion equation, or heat equation, is the following:

$$\frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial x^2}$$

## Forward Time, Center Space (FTCS)

One of the simplest finite difference methods we can derive to solve the diffusion equation is called FTCS, or Forward Time, Center Space. This method is simply a combination of two methods we have explored earlier in this class: it combines the forward Euler method in time (hence, "forward time") with the central difference method in space (hence, "centered space"). We will represent our spatial discretization with $i$ and our time discretization with $n$. 

The FTCS representation of the diffusion equation thus becomes:

$$\frac{u_i^{n+1} - u_i^n}{\Delta t} = D\frac{u^n_{i+1} - 2u_i^n + u^n_{i-1}}{\Delta x^2}$$

Take a moment to convince yourself that this is a forward Euler approximation of the time derivative and a central-difference version of the second derivative on the right. 

&#128310; Rearrange this equation to solve for $u_i^{n+1}$, and explain in your own words what this represents.

### Solving the diffusion equation

Solve the diffusion equation for initial conditions $u(t=0, x) = 5cos(\pi x)$ and boundary conditions $u(t, 0) = 2$ and $u(t, 1) = 5$, with the diffusion coefficient $D = 0.008$. These and other parameters are outlined in the comments below. 

&#128310; Before you write a single line of code, think about what the solution to this diffusion equation should look like after a long time. If it helps, visualize the problem as an unevenly heated rod (remember that this is also the 1D heat equation) with ends held at 2 and 5 degrees. Describe what you expect the solution to look like. 

&#128309; Solve the problem following the comments below.

In [ ]:
# Initialize the number of timesteps to 5000 and the number of spatial grid points to 100

# Initialize the spatial grid, which should go from 0 to 1

# Define deltax and deltat, where the total x extent is 1 and the total time extent is 30

# Set the diffusion coefficient D to 0.008

# Initialize the solution matrix u, which will be a 2D array with nt rows and nx columns

# Let's set our initial conditions as u(t=0, x) = 5cos(pi*x) and our boundary conditions as u(t, 0) = 2 and u(t, 1) = 5

# Step through time and solve the diffusion equation using the FTCS method and array indexing!


&#128309; Plot your solution (u as a function of x) every 300 timesteps or so. Experiment with the plot until you are satisfied with it. 

&#128310; Describe your plot. As time goes on, is the solution approaching what you visualized as the solution after long times?

## Intro to Numerical Instability

Repeat the diffusion experiment above, but change the diffusion coefficient from 0.008 to 0.009. Keep everything else exactly the same, including the plotting. 

In [ ]:
# Initialize the number of timesteps to 5000 and the number of spatial grid points to 100

# Initialize the spatial grid, which should go from 0 to 1

# Define deltax and deltat, where the total x extent is 1 and the total time extent is 30

# Set the diffusion coefficient D to 0.009

# Initialize the solution matrix u, which will be a 2D array with nt rows and nx columns

# Let's set our initial conditions as u(t=0, x) = 5cos(pi*x) and our boundary conditions as u(t, 0) = 2 and u(t, 1) = 5

# Step through time and solve the diffusion equation using the FTCS method and array indexing!


&#128310; Eek! Describe in your own words what you observe. 

### von Neumann Stability Analysis

You should have just observed **numerical instability** of the FTCS method applied to the diffusion equation. A numerical scheme is called *stable* if numerical noise (from initial conditions,
round-off errors, etc.), does not grow. 

There are two distinct concepts we should add to our definition of stability: 

1. A numerical scheme is *consistent* if its discrete operators converge toward the continuous operators of the PDE as $\Delta x, \Delta t \rightarrow 0$. In other words, you recover the real PDE in the limit of vanishing trunctation error. 
2. A numerical scheme is *convergent* if its solution converges towards the real solution of the PDE in the limit $\Delta x, \Delta t \rightarrow 0$.

These concepts are related by Lax's equivalence theorem, which states that *for a consistent finite difference method, for a well-posed linear initial value problem, the method is convergent if and only if it is stable.* Since we ultimately care about convergence, this theorem lets us assess convergence by assessing consistency and stability. 

To understand and predict the numerical stability of different schemes, we use **von Neumann stability analysis**, which models the numerical noise as a Fourier series. When we are solving a linear PDE, the numerical noise, which is what is added to the actual solution, obeys the discretized PDE as well. Consider solutions of the form

$$u_i^n = \sum_k A_k^n e^{-ikx_i}$$

where $k$ is a real spatial wave number and $A_k$ is the **amplification factor** at a given wave number $k$. If you plug this Fourier series into our FTCS diffusion equation (and dust off some trig identities) you will find 

$$A_k = 1 - \frac{4 D \Delta t}{(\Delta x)^2} \sin^2(\frac{k \Delta x}{2})$$

Our equations are unstable if is $\left|A_k\right| \gt 1$ for any $k$. If we impose that $\left|A_k\right| \leq 1$, we derive the **stability criterion** for this problem

$$\frac{2 D \Delta t}{(\Delta x)^2} \leq 1$$

&#128309; Compute and nicely print the stability condition for the FTCS diffusion equation for the values used above. Try for both D = 0.009 and D = 0.008. 

&#128310; Comment on the stability of the FTCS method applied to the diffusion equation. 

Methods like this, that are stable under certain circumstances, are called *conditionally stable.* 

## The advection equation

It gets worse! If we instead consider the FTCS version of the advection equation,

$$\frac{\partial u}{\partial t} = -v \frac{\partial u}{\partial x}$$

we find

$$u_i^{n+1} = u_i^n - v\frac{\Delta t}{2 \Delta x} (u_{i+1}^n - u_{i-1}^n)$$

A von Neumann stability analysis will show that this is **unconditionally unstable**! Try it at home. In fact, the FTCS method is generally unstable for hyperbolic problems. Let's explore what to do about this -- but first, let's learn a new Python skill.

## List comprehensions

A Python **list comprehension** provides a powerful way to create a list by iterating over an iterable in one line. The list comprehension is written as brackets containing an expression and a `for` clause, sometimes followed by `if` clause(s) and/or additional `for` clauses. List comprehensions can be a great tool for writing cleaner, more readable code. 

The syntax for a simple list comprehension is as follows:
`newList = [ expression(element) for element in oldList if condition ]`

For example, let's say I want a list containing $x^2$ for the numbers 0-9, but only for odd values of $x$, I could write:

`squares = [x**2 for x in range(10) if x%2 == 1]`

`%` is the modulo operator in Python: `a % b` returns the remainder of $a/b$. Try it!

In [ ]:
[x**2 for x in range(10) if x%2 == 1]

&#128309; Write a list comprehension that returns all numbers between 0 and 1000 that are divisible by 7.

&#128309; Write a list of 10 or so words -- whatever the first 10 words that come to mind are. Write a list comprehension that returns the length of each of the words using `len(string)` to find the length of each. *Optional bonus hint*: if you want to get fancy, you can create your list of 10 words using a list comprehension and a string that represents a sentence by using `string.split()`. Try it!

## The advection problem with a square wave

Let's investigate the advection problem, starting with a classic test problem for numerical schemes. Consider a single squarewave pulse defined by the condition:

$$u(x) = \begin{cases}
    1, & \text{if } 0.4 \leq x \leq 0.6\\
    0, & \text{otherwise}\\
\end{cases}$$

&#128309; Why were we talking about list comprehensions? Because an elegant way to create our square wave initial condition is to write a function `squarewave` that evaluates the above conditional, and then assign it to our $x$ domain using a list comprehension (instead of a for loop). Do that below, using a 100-element grid for x that spans values 0 to 1. Convert your list into a numpy array. Plot your results and be sure they look as expected.

### The `np.roll` function

We're going to use periodic boundary conditions for this problem, meaning that stepping off of our x domain on one side means stepping *on* to our domain from the other side. A useful numpy function is `np.roll`. Check out <a target="_blank" rel="noopener noreferrer" href="https://numpy.org/doc/stable/reference/generated/numpy.roll.html">the documentation for `np.roll`</a>. 

&#128309; Roll your squarewave pulse by 20, by 40, by 60, etc., and plot the results on the same plot as the original.

&#128310; Explain on your results and explain how `np.roll` works in your own words. How would you roll your pulse ``backward`` instead of forward?

### Applying FTCS to the advection problem for our square wave

We are finally ready to apply FTCS advection to our square wave. 

&#128310; Before writing any code, describe what you expect to see if you advect your square wave and your method works perfectly (i.e. if your method is stable and consistent).

To save you some scrolling, here is the FTCS version of the 1D advection equation again:

$$u_i^{n+1} = u_i^n - v\frac{\Delta t}{2 \Delta x} (u_{i+1}^n - u_{i-1}^n)$$

And we will use periodic boundary conditions. 

&#128309; Implement FTCS advection for your square wave following the comments below. 

In [ ]:
# Initialize the number of timesteps (nt) to 10 and the number of spatial grid points (nx) to 100

# Define deltax and deltat, where the total x extent is 1 and the total time extent is 0.1

# Define the constant v to be 1.0

# Initialize the spatial grid, which should go from 0 to 1

# Initialize the solution matrix u, which will be a 2D array with nt rows and nx columns

# Set the initial conditions of u using your squarewave function and a list comprehension

# Step through time and solve the advection equation using the FTCS method and np.roll! Plot your results for each timestep.

&#128310; Describe and explain your results in your own words. 

## The Lax Method

Let's make a slight tweak to the FTCS method: instead of using the $u_i^n$ term in the time derivative, let's use the average of the two adjacent points, i.e.:

$$u_i^n \rightarrow \frac{1}{2}\left(u_{i+1}^n + u_{i-1}^n\right)$$

If we make this replacement, the Lax method expression for the advection equation becomes

$$u_i^{n+1} = \frac{1}{2}\left(u_{i+1}^n + u_{i-1}^n\right) - v\frac{\Delta t}{2 \Delta x} (u_{i+1}^n - u_{i-1}^n)$$

&#128309; Implement Lax advection for your square wave. Use exactly the same constants and discretization as in the FTCS method version of the advection equation above, and plot your result.

In [ ]:
# Initialize the number of timesteps (nt) to 10 and the number of spatial grid points (nx) to 100

# Define deltax and deltat, where the total x extent is 1 and the total time extent is 0.1

# Define the constant v to be 1.0

# Initialize the spatial grid, which should go from 0 to 1

# Initialize the solution matrix u, which will be a 2D array with nt rows and nx columns

# Set the initial conditions of u using your squarewave function and a list comprehension

# Step through time and solve the advection equation using the Lax method and np.roll! Plot your results for each timestep.

&#128310; Explain your plot in your own words. Is this working better or worse than the FTCS method for the advection problem? How do you know?

&#128309; Use the Lax method code above, but run for 100 timesteps instead of 10 and plot.

&#128310; Describe what happens for the 100-timestep case. Experiment with other parameters in your code to understand their effects. 

What is going on? If we do the von Neumann stability analysis on the Lax implementation of the advection equation, you will find an amplification factor 

$$A_k = \cos (k \Delta x) - i \frac{v \Delta t}{\Delta x} \sin (k \Delta x)$$

And, imposing $\left|A\right|^2 \leq 1$, we find the stability criterion

$$\frac{v \Delta t}{\Delta x} \leq 1$$

This is the famous **Courant-Friedrichs-Lewy stability criterion**, also called the "CFL condition". Thus as long as your parameters satisfy this condition, the Lax implementation of the advection equation will be stable. But in your experiment above, you should have discovered that the *amplitude of your square wave decays* as time goes on. Why? Let's rewrite the Lax advection equation in just a slightly different form:

$$\frac{u_i^{n+1} - u_i^{n}}{\Delta t} = -v \left(\frac{u_{i+1}^n - u_{i-1}^n}{2\Delta x}\right) + \frac{1}{2}\left(\frac{u_{i + 1}^n - 2u_i^n + u_{i-1}^n}{\Delta t}\right)$$ 

Squint at this, and you'll realize that it's just the FTCS representation of the following equation:

$$\frac{\partial u}{\partial t} = -v \frac{\partial u}{\partial x} + \frac{(\Delta x)^2}{2\Delta t} \nabla^2 u$$

&#128310; Wait-- that's not just the advection equation any more! What type of term have we added?

This scheme is said to have **numerical dissipation**. We have developed a scheme to solve the advection equation that is *conditionally stable* at the expense of adding this numerical dissipation. This is just a taste of the wide, awesome world of numerical techniques for solving PDE systems. A key takeaway: numerical stability depends on your solver and your problem, and *must* be understood!

# Acknowledgments

S.E. Clark, with thanks to Numerical Recipes (Press et al 2007).